<font size = "5">

**Dynamic Programming**

<font size = "4">

Problem: We need to give change to a customer and want to use the minimum number of coins. What is the solution to this problem?

This is an example of an **optimization** problem.

First, we can try a **greedy algorithm**: solve the problem by making a *locally optimal* choice at each stage of the problem-solving process.

For this problem, we could use as many coins of the largest denomination as possible, then as many of the 2nd largest denomination as possible, etc.

In [7]:
def make_change_greedy(coin_denoms, change):
    coins = sorted(coin_denoms, reverse=True)
    remaining = change
    num_coins = 0
    for d in coins:
        while d <= remaining:
            num_coins += 1
            remaining -= d 
    return num_coins



In [13]:
coin_denoms = [1, 5, 10, 25]

min_coins = make_change_greedy(coin_denoms, 63)
print(min_coins)

6


<font size = "4">

For standard U.S. coins, this approach does lead to the optimal solution. To make change of 63 cents, you need 6 coins: 2 quarters, 1 dime, 3 pennies.

But if the govt. introduced a new 21 cent coin, then the greedy method would lead to a non-optimal solution.

In [14]:
coin_denoms = [1, 5, 10, 21, 25]
min_coins = make_change_greedy(coin_denoms, 63)
print(min_coins)

print(21 + 21 + 21) # only takes 3 coins

6
63


<font size = "4">

We can solve this problem recursively as follows. 

If we need to make 63 cents change (going back to standard U.S.) coins, we have 4 options:

- Use 1 quarter, and then make change for 63 - 25 = 38 cents.

- Use 1 dime, then make change for 63 - 10 = 53 cents.

- Use 1 nickel, then make change for 63 - 5 = 58 cents

- Use 1 penny, then make change for 63 - 1 = 62 cents.

Let $f(n)$ be the minimum number of coins needed to make change for $n$ cents, and suppose we can calculate it for $n \leq 62$

Then the candidate solutions for the problem are:

- $1 + f(38)$ (starting with a quarter)

- $1 + f(53)$ (starting with a dime)

- $1 + f(58)$ (starting with a nickel)

- $1 + f(62)$ (starting with a penny)

And the optimal solution is the minimum of these 4 values.

This is implemented recursively below:

In [17]:
def make_change_1(coin_denoms, change):
    if change in coin_denoms:
        return 1
    min_coins = float("inf")
    for i in [c for c in coin_denoms if c <= change]:
        num_coins = 1 + make_change_1(
            coin_denoms, change - i
        )
        min_coins = min(num_coins, min_coins)
    return min_coins

<font size = "4">

Equivalent function that doesn't use list comprehension:

In [18]:
def make_change_1a(coin_denoms, change):
    if change in coin_denoms:
        return 1
    min_coins = float("inf")
    for i in coin_denoms:
        if i > change:
            continue
        num_coins = 1 + make_change_1(
            coin_denoms, change - i
        )
        min_coins = min(num_coins, min_coins)
    return min_coins

In [19]:
coin_denoms = [1, 5, 10, 25]

print(make_change_1(coin_denoms, 63))

6


In [20]:
coin_denoms = [1, 5, 10, 25]

print(make_change_1a(coin_denoms, 63))

6


In [21]:
coin_denoms = [1, 5, 10, 21, 25]

print(make_change_1a(coin_denoms, 63))

3


<font size = "4">

This is a slow algorithm! Here is a small piece of the necessary computations when making 26 cents change:


<div style="text-align: center;">
  <img src="files/callTree.png" alt="Centered image" width = "575">
  <figcaption><font size = "1"> Miller, Randum, Yasinovskyy (Problem Solving with Algorithms and Data Structures using Python)</figcaption>
</div>

<font size = "5">

**Top-down** dynamic programming with **memoization** (not memorization)

<font size = "4">

Save intermediate results in a memorandum (memo). Also known as saving intermediate results in a **cache**. So this is sometimes called top-down dynamic programming with caching.

In [23]:
def make_change_2(coin_value_list, change, known_results):
    min_coins = change
    if change in coin_value_list:
        known_results[change] = 1
        return 1
    elif known_results[change] > 0:
        return known_results[change]
    else:
        for i in [c for c in coin_value_list if c <= change]:
            num_coins = 1 + make_change_2(coin_value_list, change - i, known_results)
            if num_coins < min_coins:
                min_coins = num_coins
            known_results[change] = min_coins
    return min_coins



In [24]:
denoms = [1, 5, 10, 25]
change = 63
result_cache = [0] * (change + 1)


print(make_change_2(denoms, change, result_cache))

6


In [25]:
print(result_cache)

[0, 1, 2, 3, 4, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 2, 3, 4, 5, 6, 2, 3, 4, 5, 6, 1, 2, 3, 4, 5, 2, 3, 4, 5, 6, 2, 3, 4, 5, 6, 3, 4, 5, 6, 7, 3, 4, 5, 6, 7, 2, 3, 4, 5, 6, 3, 4, 5, 6, 7, 3, 4, 5, 6]


In [26]:
denoms = [1, 5, 10, 25]
change = 11
result_cache = [0] * (change + 1)

sol = make_change_2(denoms, change, result_cache)


print(sol)
print(result_cache)

2
[0, 1, 0, 0, 0, 1, 2, 0, 0, 0, 1, 2]


<font size = "5">

**Bottom-up** dynamic programming

<font size = "4">

We can take a bottom-up approach, filling in optimal values for smaller change until we reach the value we are looking for. This is **not** recursive!


<div style="text-align: center;">
  <img src="files/changeTable.png" alt="Centered image" width = "375">
  <figcaption><font size = "1"> Miller, Randum, Yasinovskyy (Problem Solving with Algorithms and Data Structures using Python)</figcaption>
</div>


<div style="text-align: center;">
  <img src="files/elevenCents.png" alt="Centered image" width = "375">
  <figcaption><font size = "1"> Miller, Randum, Yasinovskyy (Problem Solving with Algorithms and Data Structures using Python)</figcaption>
</div>

In [27]:
def make_change_3(coin_value_list, change, min_coins):
   for cents in range(change + 1):
      coin_count = cents
      for j in [c for c in coin_value_list if c <= cents]:
            if min_coins[cents - j] + 1 < coin_count:
               coin_count = min_coins[cents - j] + 1
      min_coins[cents] = coin_count
   return min_coins[change]

In [28]:
denoms = [1, 5, 10, 25]
change = 11
result_cache = [0] * (change + 1)

sol = make_change_3(denoms, change, result_cache)


print(sol)
print(result_cache)

2
[0, 1, 2, 3, 4, 1, 2, 3, 4, 5, 1, 2]


<font size = "4">

Keep track of which coins were used

In [45]:
def make_change_4(coin_value_list, change, min_coins, coins_used):
    for cents in range(change + 1):
        coin_count = cents
        new_coin = 1
        for j in [c for c in coin_value_list if c <= cents]:
            if min_coins[cents - j] + 1 < coin_count:
                coin_count = min_coins[cents - j] + 1
                new_coin = j
        min_coins[cents] = coin_count
        coins_used[cents] = new_coin
    return min_coins[change]


def print_coins(coins_used, change):
    coin = change
    while coin > 0:
        this_coin = coins_used[coin]
        print(this_coin, end=" ")
        coin = coin - this_coin
    print()

In [46]:
amnt = 63
clist = [1, 5, 10, 21, 25]
coins_used = [0] * (amnt + 1)
coin_count = [0] * (amnt + 1)

print(
       "Making change for {} requires the following {} coins: ".format(
             amnt, make_change_4(clist, amnt, coin_count, coins_used)
       ),
       end="",
    )

print_coins(coins_used, amnt)
print("The used list is as follows:")
print(coins_used)

Making change for 63 requires the following 3 coins: 21 21 21 
The used list is as follows:
[1, 1, 1, 1, 1, 5, 1, 1, 1, 1, 10, 1, 1, 1, 1, 5, 1, 1, 1, 1, 10, 21, 1, 1, 1, 25, 1, 1, 1, 1, 5, 10, 1, 1, 1, 10, 1, 1, 1, 1, 5, 10, 21, 1, 1, 10, 21, 1, 1, 1, 25, 1, 10, 1, 1, 5, 10, 1, 1, 1, 10, 1, 10, 21]


In [47]:
amnt = 6
clist = [1, 5, 10, 21, 25]
coins_used = [0] * (amnt + 1)
coin_count = [0] * (amnt + 1)

print(
       "Making change for {} requires the following {} coins: ".format(
             amnt, make_change_4(clist, amnt, coin_count, coins_used)
       ),
       end="",
    )

print_coins(coins_used, amnt)
print("The used list is as follows:")
print(coins_used)

Making change for 6 requires the following 2 coins: 1 5 
The used list is as follows:
[1, 1, 1, 1, 1, 5, 1]
